In [34]:
# Imports and pip installations (if needed)
import pandas as pd
from sklearn.preprocessing import MinMaxScaler



# Part 1: Load the dataset

In [ ]:
# Load the given datasets
categorical_df = pd.read_csv('chronic_kidney_disease_categorical.csv')
numerical_df = pd.read_csv('chronic_kidney_disease_numerical.csv')

# Print the data
categorical_df.head(), numerical_df.head()


# Part 2: Analyze the Dataset

Refer to this: https://archive.ics.uci.edu/dataset/336/chronic+kidney+disease

Explain what the each data is in your own words. What are the features and labels? Are the features in the given datasets : categorical, numerical or both? Give 3 examples of categorical and numerical columns each (if they exist)

Answer: 

The features are the various attributes that are used to predict whether the patient has chronic kidney disease, these include blood pressure, specific gravity etc. The dataset also mentions the type of data, whether it's an integer, categorical, binary and its units. These are used as the variables and parameters used when considering if someone has the disease or not. 

The label is either ckd, notckd, which represents the target variable: whether the patient has the chronic kidney disease or not.

Age, Blood Pressure, Blood Glucose Random, Blood Urea Serum Creatinine, Sodium, Potassium, Hemoglobin, Packed cell volume, White blood cell count, red blood cell count – These are all the numerical columns of the dataset.

Albumin, sugar are both categorical and numerical.

Red blood cell, pus cell, pus cell clumps, bacteria, hypertension, diabetes mellitus, coronary artery disease, appetite, pedal edema, anemia – Thse are all the categorical columns.



# Part 3: Data Preprocessing

A fundamental skill in Machine Learning is mastering the art of data cleaning and preprocessing. In this assignment, you will learn and apply essential data cleaning techniques to transform a raw dataset into a clean, ready-to-use form which you can use for regression or classification tasks. By the end of this assignment, you'll have a fully clean dataset and a solid foundation in preparing data for various machine learning models.

## Part 3.1 : Drop Duplicate rows

Let's start by checking if the given datasets have any duplicate rows (same Unique Id). Use pandas to identify and remove these duplicate rows from the given dataset

In [ ]:
# For the numerical dataset, check if there are duplicate rows in the dataset. If yes, print total number of duplicate rows
num_duplicates = numerical_df.duplicated(subset='unique_id').sum()
print(num_duplicates)

# Drop these duplicate rows
num_df_noduplicates = numerical_df.drop_duplicates(subset='unique_id')

# Repeat the same for categorical dataset. Print the duplicate rows and drop them
cat_duplicates = categorical_df.duplicated(subset='unique_id').sum()
print(cat_duplicates)

cat_df_noduplicates = categorical_df.drop_duplicates(subset='unique_id')


## Part 3.2: Combine two differents datasets

A good skill to have is to know how to combine 2 different datasets.

Are all the unique ids are present in both datasets? Why do you think so? If not, what do the rows that are missing from one of the datasets look like in the combined table?

Answer:

In [ ]:
# Merge the two given numerical and categorical datasets based on their unique_ID.
combined_df = pd.merge(num_df_noduplicates, cat_df_noduplicates,  on='unique_id', how='outer')

#Print the combined dataset
combined_df


## Part 3.3: Rows with Missing values

Removing missing values from a dataset is important for classification because it ensures the model is trained on complete and accurate data, leading to better performance and reliable predictions. Incomplete data can introduce bias and errors, negatively impacting the model's effectiveness.

In [ ]:
# Calculate the percentage of rows that contain atleast one missing value
rows_atleast1missing = combined_df.isnull().any(axis=1).sum()
total_rows = combined_df.shape[0]
percentage_missing = (rows_atleast1missing / total_rows) * 100

# Print %
percentage_missing = float(percentage_missing)
percentage_missing

# Drop these rows from the dataset
combined_df_noNaN = combined_df.dropna()

# Print the Dataset
combined_df_noNaN

## Part 3.4: Sort the dataset according to the Labels

In [ ]:
# Sort the dataset according to the values in 'Target' column. Make sure reset the indices after sorting
sorted_target_df = combined_df_noNaN.sort_values(by='Target')
sorted_reset_df = sorted_target_df.reset_index(drop=True)

# Print the dataset
sorted_reset_df

## Part 3.5: Encoding Categorical data

In this step, we identify and process the categorical columns in the sorted dataset. We map each unique value in these columns to separate "dummy" columns.

For example, the column 'rbc' will be transformed into two columns 'rbc_normal' and 'rbc_abnormal'. If a row's value in 'rbc' is 'normal', the 'rbc_normal' column will be set to 1 and 'rbc_abnormal' will be set to 0.


**Note: Find a correct pandas function to do this **

In [ ]:
# Write code here
categorical_columns = ['rbc', 'pc', 'pcc', 'ba', 'htn', 'dm', 'cad', 'appet', 'pe', 'ane', 'Target']

encoded_df = pd.get_dummies(sorted_target_df, columns=categorical_columns).astype(int)

# Print the dataset
encoded_df

In the example we went through above, another solution is to have a single column for the binary variable. In the downstream modeling would this be equivalent? What effect would this have if the categorical variable could take more than 2 values? For example, let's say we have a categorical feature that is "type of condiment" that can take 5 separate values and we are trying to predict the rating of a particular sandwich.

## Part 3.6 : Remove Outliers from Numerical Columns

Outliers can disproportionately influence the fit of a regression model, potentially leading to a model that does not generalize well therefore it is important that we remove outliers from the numerical columns of the dataset.

For this dataset, we define an outlier to be 3 times the standard deviation from the mean. Drop these outliers from the dataset

In [ ]:
# Remove outliers
numerical_columns = ['age', 'bp', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hemo', 'pcv', 'wbcc', 'rbcc']

means = {}
std_devs = {}

for column in numerical_columns:
    means[column] = encoded_df[column].mean()
    std_devs[column] = encoded_df[column].std()

no_outliers_df = encoded_df.copy()
for column in numerical_columns:
    lower_bound = means[column] - 3 * std_devs[column]
    upper_bound = means[column] + 3 * std_devs[column]
    no_outliers_df = no_outliers_df[(no_outliers_df[column] >= lower_bound) & (no_outliers_df[column] <= upper_bound)]

# Print the dataset
no_outliers_df

## Part 3.7 : Normalize the Numerical Columns

Normalizing numerical attributes ensures that all features contribute equally to the model by scaling them to a consistent range, which improves model performance and convergence. It prevents features with larger scales from disproportionately influencing the model's learning process.

In [ ]:
scale = MinMaxScaler()

normalized_df = no_outliers_df.copy()

normalized_df[numerical_columns] = scale.fit_transform(normalized_df[numerical_columns])

# Print the dataset
normalized_df

## Part 3.8: Remove Unnecessary columns

Are there any columns in this dataset which are not appropriate for modeling and predictions? Which column(s)? Justify their exclusion and remove them

Answer:

In [ ]:
#Remove that column
normalized_sorted_df = normalized_df.sort_values(by='unique_id').reset_index(drop=True)

new_normalized_sorted_df = normalized_sorted_df.drop(columns=['unique_id'])

# Print the dataset
new_normalized_sorted_df

## Part 3.9: Export the Cleaned Data

Now that you've completed these cleaning steps you should have a pandas dataframe which is much cleaner and ready for modeling. Our final step is to save our work. Export the DataFrame to a two new formats: csv and json.

In [44]:
# Export the dataframe to a new csv file
new_normalized_sorted_df.to_csv('cleaned_kidney_disease_data.csv', index=False)

# Save as JSON
new_normalized_sorted_df.to_json('cleaned_kidney_disease_data.json', orient='records')



# Part 4: Data conversions with Large Language Models

One powerful use case of ChatGPT (and other generative language models) is cleaning and transforming data. In some cases, these models can directly manipulate loosely structured data that you provide to them into a standard format. In the other cases, you can often prompt the model to create a conversion or extraction script for you in python or Pandas and then run it on your own. 

In this part of the assignment you will prompt 383GPT to explore these capabilities.

## Part 4.1 GPT Data Manipulation

Take the cleaned dataset that you created in part three and output the top 15 rows of that dataset. Then copy the terminal output, open 383gpt and ask it to convert that output to a markdown table. Paste that markdown table in the cell bellow

In [ ]:
top_15_rows = new_normalized_sorted_df.head(15)
top_15_rows


### Paste the markdown table here

|    |       age |    bp |       bgr |       bu |     sc |      sod |    pot |      hemo |      pcv |      wbcc | ... |   cad_no |   cad_yes |   appet_good |   appet_poor |   pe_no |   pe_yes |   ane_no |   ane_yes |   Target_ckd |   Target_notckd |
|----|-----------|-------|-----------|----------|--------|----------|--------|------------|----------|-----------|-----|----------|-----------|---------------|---------------|---------|----------|----------|-----------|---------------|-----------------|
|  0 | 0.783784  |  0.00 | 0.154506  | 0.111111 | 0.000  | 1.000000 | 0.25   | 0.750000   | 0.684211 | 0.306452  | ... |        1 |         0 |             1 |             0 |       1 |        0 |        1 |         0 |             0 |                 1 |
|  1 | 0.472973  |  0.50 | 0.223176  | 0.098039 | 0.000  | 0.600000 | 0.75   | 1.000000   | 0.657895 | 0.387097  | ... |        1 |         0 |             1 |             0 |       1 |        0 |        1 |         0 |             0 |                 1 |
|  2 | 0.486486  |  0.25 | 0.098712  | 0.143791 | 0.000  | 0.766667 | 0.50   | 0.916667   | 0.710526 | 0.225806  | ... |        1 |         0 |             1 |             0 |       1 |        0 |        1 |         0 |             0 |                 1 |
|  3 | 0.918919  |  0.00 | 0.077253  | 0.261438 | 0.000  | 0.900000 | 0.25   | 1.000000   | 0.973684 | 0.137097  | ... |        1 |         0 |             1 |             0 |       1 |        0 |        1 |         0 |             0 |                 1 |
|  4 | 0.851351  |  0.25 | 0.832618  | 0.503268 | 0.250  | 0.333333 | 0.50   | 0.583333   | 0.552632 | 0.427419  | ... |        0 |         1 |             1 |             0 |       0 |        1 |        1 |         0 |             1 |                 0 |
|  5 | 0.418919  |  0.00 | 0.167382  | 0.241830 | 0.125  | 0.700000 | 0.50   | 0.833333   | 0.842105 | 0.217742  | ... |        1 |         0 |             1 |             0 |       1 |        0 |        1 |         0 |             0 |                 1 |
|  6 | 0.567568  |  0.50 | 0.270386  | 0.843137 | 1.000  | 0.400000 | 0.75   | 0.416667   | 0.526316 | 0.153226  | ... |        1 |         0 |             1 |             0 |       0 |        1 |        1 |         0 |             1 |                 0 |
|  7 | 0.621622  |  0.50 | 0.236052  | 0.078431 | 0.125  | 0.633333 | 0.50   | 0.916667   | 0.710526 | 0.032258  | ... |        1 |         0 |             1 |             0 |       1 |        0 |        1 |         0 |             0 |                 1 |
|  8 | 0.310811  |  0.50 | 0.055794  | 0.254902 | 0.000  | 0.633333 | 0.25   | 1.000000   | 0.631579 | 0.451613  | ... |        1 |         0 |             1 |             0 |       1 |        0 |        1 |         0 |             0 |                 1 |
|  9 | 0.905405  |  1.00 | 0.965665  | 0.522876 | 0.625  | 0.666667 | 0.00   | 0.333333   | 0.368421 | 0.217742  | ... |        0 |         1 |             0 |             1 |       1 |        0 |        1 |         0 |             1 |                 0 |
| 10 | 0.837838  |  0.00 | 0.236052  | 0.202614 | 0.125  | 0.633333 | 0.25   | 1.000000   | 0.894737 | 0.193548  | ... |        1 |         0 |             1 |             0 |       1 |        0 |        1 |         0 |             0 |                 1 |
| 11 | 0.716216  |  0.00 | 0.184549  | 0.084967 | 0.125  | 0.633333 | 0.25   | 0.833333   | 1.000000 | 0.177419  | ... |        1 |         0 |             1 |             0 |       1 |        0 |        1 |         0 |             0 |                 1 |
| 12 | 0.567568  |  0.00 | 0.180258  | 0.222222 | 0.125  | 0.733333 | 0.50   | 0.750000   | 0.736842 | 0.411290  | ... |        1 |         0 |             1 |             0 |       1 |        0 |        1 |         0 |             0 |                 1 |
| 13 | 0.810811  |  0.25 | 0.158798  | 0.039216 | 0.125  | 0.666667 | 0.25   | 0.666667   | 0.684211 | 0.540323  | ... |        1 |         0 |             1 |             0 |       1 |        0 |        1 |         0 |             0 |                 1 |
| 14 | 0.608108  |  0.00 | 0.124464  | 0.183007 | 0.000  | 0.500000 | 0.25   | 0.666667   | 0.868421 | 0.322581  | ... |        1 |         0 |             1 |             0 |       1 |        0 |        1 |         0 |             0 |                 1 |


** Caution: ** while language models can perform data conversions they also can * hallucinate * during this process, particularly for bigger datasets. Reflect on this below, how could you mitigate data conversion hallucinations from LLM conversions?

When using LLMs like 383 gpt for data conversion, they might "hallucinate" especially when it comes to larger data sets. In mine particularly, since there were only 15 rows to be marked down it worked and it was pretty straightforward and simple. In fact I think it is worthwhile for quicker conversions and formatting for small datasets, but not for larger or highly precise conversions. To mitigate these hallucinations, chunking the data is a potential strategy. Converting the data into smaller manageable chunks can reduce risk of hallucination. 


## Part 4.2 GPT Pandas Prompting

In this section, you will prompt 383GPT to write pandas code manipulations for you.

After working with this data for awhile, we realized we're starting to forget the meanings of the abbreviated column names. Let's ask 383GPT to fix this for us. First, navigate to the [UCI dataset overview](https://archive.ics.uci.edu/dataset/336/chronic+kidney+disease) and copy the abbrevation to name mapping. Then, go to 383GPT and instruct the LLM to provide you with a pandas script to apply this renaming to all the columns of your dataset. Paste that code below and make any adjustments necessary to run it in your notebook.

In [ ]:
# Code to rename all the columns in the dataset
column_mapping = {
    'age': 'Age',
    'bp': 'Blood Pressure',
    'bgr': 'Blood Glucose Random',
    'bu': 'Blood Urea',
    'sc': 'Serum Creatinine',
    'sod': 'Sodium',
    'pot': 'Potassium',
    'hemo': 'Hemoglobin',
    'pcv': 'Packed Cell Volume',
    'wbcc': 'White Blood Cell Count',
    'rbcc': 'Red Blood Cell Count',
    'rbc_abnormal': 'Red Blood Cell Abnormal',
    'rbc_normal': 'Red Blood Cell Normal',
    'pc_abnormal': 'Pus Cell Abnormal',
    'pc_normal': 'Pus Cell Normal',
    'pcc_notpresent': 'Pus Cell Clumps Not Present',
    'pcc_present': 'Pus Cell Clumps Present',
    'ba_notpresent': 'Bacteria Not Present',
    'ba_present': 'Bacteria Present',
    'htn_no': 'Hypertension No',
    'htn_yes': 'Hypertension Yes',
    'dm_no': 'Diabetes Mellitus No',
    'dm_yes': 'Diabetes Mellitus Yes',
    'cad_no': 'Coronary Artery Disease No',
    'cad_yes': 'Coronary Artery Disease Yes',
    'appet_good': 'Appetite Good',
    'appet_poor': 'Appetite Poor',
    'pe_no': 'Pedal Edema No',
    'pe_yes': 'Pedal Edema Yes',
    'ane_no': 'Anemia No',
    'ane_yes': 'Anemia Yes',
    'Target_ckd': 'Chronic Kidney Disease',
    'Target_notckd': 'No Chronic Kidney Disease'
}

new_normalized_sorted_df.rename(columns=column_mapping, inplace=True)

new_normalized_sorted_df

## Part 4.3 Augmenting our skills with prompting

In addition, we can also use 383GPT to convert our data manipulation operations between different data manipulation languages and libraries. For example let's prompt 383GPT to convert the following SQL query to a pandas query.

**SQL Query**
```sql
SELECT Target, COUNT(*) AS count
FROM your_table_name
GROUP BY Target;
```

Prompt 383GPT to convert this to a pandas query. Run this query below, then describe what it does. (If you're not familiar with SQL that is okay you need to only comment on the final resulting output.)

In [ ]:
# Converted SQL to Pandas code
target_count = new_normalized_sorted_df.groupby('Chronic Kidney Disease').size().reset_index(name='count')
target_count


*Describe what the above code does here*

The .groupby('Chronic Kidney Disease') function groups the dataset by the target column (Target_ckd). Then, the .size() counts the number of rows in each group. Lastly, the .reset_index(name='count') creates a new dataframe where the counts are shown in a column. Finally, the output shows us how many rows in the dataset correspond to patients with the disease (CKD) and how many don't have the disease. 